## MLflow's Model Registry

In [1]:
import mlflow

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.get_experiment_by_name("nyc-taxi-experiment")

<Experiment: artifact_location=('file:///d:/Universidad/Cursos/MLOps en '
 'Zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1748017795294, experiment_id='1', last_update_time=1748017795294, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
# Muestra los experimentos en el servidor por id
client.search_experiments()

[<Experiment: artifact_location=('file:///d:/Universidad/Cursos/MLOps en '
  'Zoomcamp/02-experiment-tracking/mlruns/2'), creation_time=1748040190626, experiment_id='2', last_update_time=1748040190626, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location=('file:///d:/Universidad/Cursos/MLOps en '
  'Zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1748017795294, experiment_id='1', last_update_time=1748017795294, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location=('file:///d:/Universidad/Cursos/MLOps en '
  'Zoomcamp/02-experiment-tracking/mlruns/0'), creation_time=1748017795150, experiment_id='0', last_update_time=1748017795150, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
# Crear un nuevo experimento
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [17]:
# Aqui se busca los runs de un experimento por id filtrando por rmse menor a 7
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.4",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [18]:
# Se listan los runs encontrados
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: e5e7630a78574183aa8a1912f435831f, rmse: 6.3064
run id: cdabe3a537e94d28a795ed338d3764a9, rmse: 6.3184
run id: 8050386818d14521bb3a9461b8c690f0, rmse: 6.3897


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [46]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
# Para registrar un nuevo run en codigo
#run_id = "b4b6d5e2bfe546c9830efdfaae087b14"
#model_uri = f"runs:/{run_id}/model"
#mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [31]:
model_name = "nyc-taxi-regressor"
# Asignar alias "staging" a la versión 1
client.set_registered_model_alias(model_name, "staging", 1)

# Asignar alias "production" a la versión 2
client.set_registered_model_alias(model_name, "production", 2)

In [32]:
aliases = client.get_model_version_by_alias(model_name, "production")
print(aliases)

<ModelVersion: aliases=['production'], creation_timestamp=1748038343719, current_stage='None', description='', last_updated_timestamp=1748038343719, name='nyc-taxi-regressor', run_id='e5e7630a78574183aa8a1912f435831f', run_link='', source=('file:///d:/Universidad/Cursos/MLOps en '
 'Zoomcamp/02-experiment-tracking/mlruns/1/e5e7630a78574183aa8a1912f435831f/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'gbr(example)'}, user_id=None, version=2>


In [33]:
client.set_model_version_tag(model_name, 1, "stage", "production")
client.set_model_version_tag(model_name, 2, "stage", "production")

In [ ]:
## Muestra la ultima version de un modelo en un stage DEPRECATED
# model_name = "nyc-taxi-regressor"
# latest_versions = client.get_latest_versions(name=model_name)

# for version in latest_versions:
#     print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


C:\Users\Daniel Muñoz Viveros\AppData\Local\Temp\ipykernel_6016\431351333.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:
## Cambiar el stage de un modelo DEPRECATED
# model_version = 4
# new_stage = "Staging"
# client.transition_model_version_stage(
#     name=model_name,
#     version=model_version,
#     stage=new_stage,
#     archive_existing_versions=False
# )

<ModelVersion: creation_timestamp=1652971637398, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972141519, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [ ]:
model_version = 2
new_stage = "Production"

In [ ]:
from datetime import datetime

date = datetime.today().date()

## Cambiar la descripcion de un modelo relacionado a un stage
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['production'], creation_timestamp=1748038343719, current_stage='None', description='The model version 2 was transitioned to Production on 2025-05-23', last_updated_timestamp=1748046029519, name='nyc-taxi-regressor', run_id='e5e7630a78574183aa8a1912f435831f', run_link='', source=('file:///d:/Universidad/Cursos/MLOps en '
 'Zoomcamp/02-experiment-tracking/mlruns/1/e5e7630a78574183aa8a1912f435831f/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'gbr(example)', 'stage': 'production'}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [ ]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, tag_stage, X_test, y_test):
#     model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
#     y_pred = model.predict(X_test)
#     return {"rmse": root_mean_squared_error(y_test, y_pred)}
    model = mlflow.pyfunc.load_model(f"models:/{name}@{tag_stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}


In [48]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [49]:
run_id = "e5e7630a78574183aa8a1912f435831f"

In [ ]:
# Esta linea descarga archivo preprocessor.db en el directorio actual en una carpeta llamada preprocessor
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'd:\\Universidad\\Cursos\\MLOps en Zoomcamp\\02-experiment-tracking\\preprocessor'

In [51]:
## Leemos el archivo preprocessor.b
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [52]:
X_test = preprocess(df, dv)

In [53]:
target = "duration"
y_test = df[target].values

In [56]:
model_name = "nyc-taxi-regressor"
tag_stage = "production"

In [ ]:
# %time test_model(name=model_name, tag_stage="production", X_test=X_test, y_test=y_test)

TypeError: test_model() got an unexpected keyword argument 'stage'

In [ ]:
#DEPRECATED
# client.transition_model_version_stage(
#     name=model_name,
#     version=4,
#     stage="Production",
#     archive_existing_versions=True
# )

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>